In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Get the whole html source from the web page and send to the BS4 html parser

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

r = requests.get(url)

html_doc = r.text

soup = BeautifulSoup(html_doc, 'html.parser')

Now get the first table, the one with the information we need.

Go through each row in the html table. If it is a header (th cells) then ignore. add the postcode, borough and neighbourhood to the results.

In [5]:
# get first table
table = soup.find("table")

rows=[]

for row in table.find_all("tr"):
    if row.find("th"):
        pass
    else:
        tds = row.find_all("td")
        postcode = tds[0].text
        borough = tds[1].text
        neighbourhood = tds[2].text.replace("\n","")
        rows.append([postcode,borough,neighbourhood])


The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [6]:
df = pd.DataFrame(data=rows,columns=['PostalCode', 'Borough', 'Neighborhood'])

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [7]:
df = df.set_index("Borough")
df.drop("Not assigned", axis=0, inplace=True)

In [8]:
df.reset_index(inplace=True)
df = df[['PostalCode','Borough','Neighborhood']]
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [9]:
df[df.Neighborhood=='Not assigned'].head()

,PostalCode,Borough,Neighborhood
6,M7A,Queen's Park,Not assigned


In [10]:
mask = df['Neighborhood'] == 'Not assigned'
df.loc[mask, 'Neighborhood'] = df['Borough']
df['Neighborhood'] = df['Neighborhood'].mask(mask, df['Borough'])
print(df.shape)
df.head(10)

(211, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [11]:
df[df.Borough=="Queen's Park"]

,PostalCode,Borough,Neighborhood
6,M7A,Queen's Park,Queen's Park


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [12]:
df = df.groupby(['PostalCode', 'Borough']).agg({'Neighborhood':lambda x: ', '.join(x)}).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [13]:
df[df.PostalCode=='M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [14]:
df.shape

(103, 3)

In [ ]:
df.to_csv('Toronto_3cols.csv')